Hier werden alle Daten aus verschiedenen Quellen zu einem großen Dataframe zusammen gefasst. Das sind zum einen die Daten welche uns vom Kurs zu Verfügung gestellt wurden, als auch die daten welche wir selber noch gefunden haben.

In [1]:
#packete für data handling laden
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
#Einlesen der Basisidaten, welche wir zur verfügung gestellt bekommen haben
#Wetterdaten
wetter = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/07_Wetter/07_wetter.csv")
print(wetter.head())
print(wetter.tail())
print(wetter.shape)

#Umsatzdaten
umsatz = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/umsatzdaten_gekuerzt.csv")
print(umsatz.head())
print(umsatz.tail())
print(umsatz.shape)

#Kieler Woche Daten
kiwo = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/kiwo.csv")
print(kiwo.head())
print(kiwo.tail())
print(kiwo.shape)

        Datum  Temp_Very_Cold  Temp_Cold  Temp_Mild  Temp_Warm  Temp_Hot  \
0  2012-01-01               0          1          0          0         0   
1  2012-01-02               0          1          0          0         0   
2  2012-01-03               0          1          0          0         0   
3  2012-01-04               0          1          0          0         0   
4  2012-01-05               0          1          0          0         0   

   Cloud_Clear  Cloud_Partly_Cloudy  Cloud_Cloudy  Wind_Light  Wind_Moderate  \
0            0                    0             1           0              1   
1            0                    0             1           0              1   
2            0                    0             1           0              0   
3            0                    1             0           0              0   
4            0                    0             1           0              0   

   Wind_Strong  Weather_Good  Weather_Light_Issues  Weather_Mo

Die Umsatzdaten sind im long format nach Warengruppen. Bedeutet, dass jedes Datum mehrmals vorkommt. Das ist zum mergen der Daten nicht ideal. Daher den dataframe vom long format ins wide format bringen. 
In diesem Zuge werden direkt auch die Warengruppen richtig benannt und eine Spalte mit den Wochentagen eingefügt

In [3]:
#get the umsatz data in the wide format
# Pivot the DataFrame
umsatz_shaped = umsatz.pivot(index='Datum', columns='Warengruppe', values='Umsatz')

# Reset the index to make 'date' a column again
umsatz_shaped.reset_index(inplace=True)

#get colums names right
umsatz_shaped.columns = ['Datum', 'Brot', 'Broetchen', 'Croissant', 'Konditorei', 'Kuchen', 'Saisonbrot']


print(umsatz_shaped.head())
print(umsatz_shaped.tail())    
print(umsatz_shaped.shape)

        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
           Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
1814  2018-07-27  198.225523  677.415844  333.774099   65.563734  297.867853   
1815  2018-07-28  214.470393  716.576825  371.819770   54.047422  290.024007   
1816  2018-07-29  129.801991  721.822447  433.820697  128.952852  335.823557   
1817  2018-07-30  148.541340  616.358562  348.770346   71.822977  324.965348   
1818  2018-07-31  123.075682  586.081666 

In [4]:
#aufräumen der Daten. NaN durch 0 ersetzen
umsatz_shaped.fillna(0, inplace=True)

#Eine Spalte hinzufügen in welcher der Gesamtumsatz steht
umsatz_shaped['Umsatz_total'] = umsatz_shaped['Brot'] + umsatz_shaped['Broetchen'] + umsatz_shaped['Croissant'] + umsatz_shaped['Konditorei'] + umsatz_shaped['Kuchen'] + umsatz_shaped['Saisonbrot']
umsatz_shaped['Umsatz_total'] = umsatz_shaped['Umsatz_total'].astype(int)


print(umsatz_shaped.head())
print(umsatz_shaped.shape)

        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  
0         0.0          1269  
1         0.0          1430  
2         0.0          1124  
3         0.0          1184  
4         0.0          1288  
(1819, 8)


In [5]:
#nun können die Umsatzdaten mit den Wetterdaten zusammengeführt werden
#merge the data
data = pd.merge(umsatz_shaped,wetter, how='left', on='Datum')
print(data.head())
print(data.shape)

#es werden nun die Zeilen behalten, für welche es Umsatzdaten gibt

        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Temp_Very_Cold  Temp_Cold  ...  \
0         0.0          1269             0.0        0.0  ...   
1         0.0          1430             0.0        0.0  ...   
2         0.0          1124             0.0        0.0  ...   
3         0.0          1184             0.0        0.0  ...   
4         0.0          1288             0.0        0.0  ...   

   Cloud_Partly_Cloudy  Cloud_Cloudy  Wind_Light  Wind_Moderate  Wind_Strong  \
0                  0.0           1.0         0.0            1.0          0.0  

In [6]:
# und nun noch die Kieler Woche Daten hinzufügen
data = pd.merge(data, kiwo, how='left', on='Datum')
print(data.head())
print(data.shape)

#im moment steht da eine 1.0 wenn es Kieler Woche ist und eine NaN wenn nicht. Das ändern wir jetzt
data['KielerWoche'] = data['KielerWoche'].fillna(0)
data['KielerWoche'] = data['KielerWoche'].astype(int)
print(data.head())	

        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Temp_Very_Cold  Temp_Cold  ...  Cloud_Cloudy  \
0         0.0          1269             0.0        0.0  ...           1.0   
1         0.0          1430             0.0        0.0  ...           0.0   
2         0.0          1124             0.0        0.0  ...           1.0   
3         0.0          1184             0.0        0.0  ...           1.0   
4         0.0          1288             0.0        0.0  ...           0.0   

   Wind_Light  Wind_Moderate  Wind_Strong  Weather_Good  Weather_Light_Iss

In [9]:
#als letztes werden noch die Wochentage hinzugefügt
wochentag = pd.read_csv("./09_Wochentage/Wochentage.csv")
print(wochentag.head())

#mergen der Daten
data = pd.merge(data, wochentag, how='left', on='Datum')
print(data.head())

        Datum  Montag  Dienstag  Mittwoch  Donnerstag  Freitag  Samstag  \
0  2013-07-01       1         0         0           0        0        0   
1  2013-07-02       0         1         0           0        0        0   
2  2013-07-03       0         0         1           0        0        0   
3  2013-07-04       0         0         0           1        0        0   
4  2013-07-05       0         0         0           0        1        0   

   Sonntag  
0        0  
1        0  
2        0  
3        0  
4        0  
        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saiso

Nun haben wir alle Basisidaten zusammen geführt. Jetzt können noch die Variablen, welche wir selber gesucht haben hinzugefügt werden.

In [10]:
#hinzufügen der Inflationsdaten
#einlesen der Daten
inflation = pd.read_csv("./02_Verbraucherpreisindex/vpi_daily.csv")
print(inflation.head())
print(inflation.tail())
print(inflation.shape)


        Datum   VPI
0  2013-07-01  98.9
1  2013-07-02  98.9
2  2013-07-03  98.9
3  2013-07-04  98.9
4  2013-07-05  98.9
           Datum    VPI
1852  2018-07-27  104.4
1853  2018-07-28  104.4
1854  2018-07-29  104.4
1855  2018-07-30  104.4
1856  2018-07-31  104.4
(1857, 2)


In [11]:
#mergen der Daten
data = pd.merge(data, inflation, how='left', on='Datum')    
print(data.head())
print(data.shape)

        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Temp_Very_Cold  Temp_Cold  ...  Weather_Unknown  \
0         0.0          1269             0.0        0.0  ...              0.0   
1         0.0          1430             0.0        0.0  ...              1.0   
2         0.0          1124             0.0        0.0  ...              0.0   
3         0.0          1184             0.0        0.0  ...              0.0   
4         0.0          1288             0.0        0.0  ...              1.0   

   KielerWoche  Montag  Dienstag  Mittwoch  Donnerstag  

In [12]:
#hinzufügen der Kreuzfahrschfissdaten
schiffe = pd.read_csv("./01_Kreuzfahrtschiffe/Alle_Schiffe.csv")
print(schiffe.head())


  Arival_date  Number_of_ships  Ship
0  2012-01-07                2  True
1  2012-01-09                2  True
2  2012-02-05                1  True
3  2012-02-06                2  True
4  2012-02-07                1  True


In [13]:
#merge the data
data = pd.merge(data, schiffe, how='left', left_on='Datum', right_on='Arival_date')
#delete extra column Arival_date
data.drop('Arival_date', axis=1, inplace=True)
#fill NaN with False
data['Ship'] = data['Ship'].fillna(False)

print(data.head())
print(data.shape)

        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Temp_Very_Cold  Temp_Cold  ...  Montag  Dienstag  \
0         0.0          1269             0.0        0.0  ...       1         0   
1         0.0          1430             0.0        0.0  ...       0         1   
2         0.0          1124             0.0        0.0  ...       0         0   
3         0.0          1184             0.0        0.0  ...       0         0   
4         0.0          1288             0.0        0.0  ...       0         0   

   Mittwoch  Donnerstag  Freitag  Samstag  Sonntag

/tmp/ipykernel_3214/2747539301.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Ship'] = data['Ship'].fillna(False)


In [14]:
#importieren der heimspiel daten
heimspiele = pd.read_csv("./03_Heimspiele/Kiel_Heimspiele.csv", sep=',')

print(heimspiele.head())
print(heimspiele.shape)

        Datum  Heimspiel
0  2013-01-01          0
1  2013-01-02          0
2  2013-01-03          0
3  2013-01-04          0
4  2013-01-05          0
(2922, 2)


In [15]:
#mergen der heimspiel Daten
data = pd.merge(data, heimspiele, how='left', left_on='Datum', right_on='Datum')
print(data.head())
print(data.shape)

        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Temp_Very_Cold  Temp_Cold  ...  Dienstag  \
0         0.0          1269             0.0        0.0  ...         0   
1         0.0          1430             0.0        0.0  ...         1   
2         0.0          1124             0.0        0.0  ...         0   
3         0.0          1184             0.0        0.0  ...         0   
4         0.0          1288             0.0        0.0  ...         0   

   Mittwoch  Donnerstag  Freitag  Samstag  Sonntag   VPI  Number_of_ships  \
0         0          

In [16]:
#Feiertagsdaten hochladen
feiertage = pd.read_csv("./04_Feiertage/feiertage_final.csv", sep=';')
print(feiertage.head())
print(feiertage.shape)

        Datum  Feiertag
0  2013-07-01         0
1  2013-07-02         0
2  2013-07-03         0
3  2013-07-04         0
4  2013-07-05         0
(1645, 2)


In [17]:
#und mergen der feiertagsdaten
data = pd.merge(data, feiertage, how='left', left_on='Datum', right_on='Datum')
print(data.head())
print(data.shape)

        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Temp_Very_Cold  Temp_Cold  ...  Mittwoch  \
0         0.0          1269             0.0        0.0  ...         0   
1         0.0          1430             0.0        0.0  ...         0   
2         0.0          1124             0.0        0.0  ...         1   
3         0.0          1184             0.0        0.0  ...         0   
4         0.0          1288             0.0        0.0  ...         0   

   Donnerstag  Freitag  Samstag  Sonntag   VPI  Number_of_ships   Ship  \
0           0        0  

In [18]:
#hinzufügen der Feriendaten
ferien = pd.read_csv("./06_Ferientage/06_ferientage.csv")
print(ferien.head())

#und mergen der Feriendaten
data = pd.merge(data, ferien, how='left', left_on='Datum', right_on='Datum')
print(data.head())

        Datum  is_holiday
0  2013-01-01           0
1  2013-01-02           0
2  2013-01-03           0
3  2013-01-04           0
4  2013-01-05           0
        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Temp_Very_Cold  Temp_Cold  ...  Donnerstag  \
0         0.0          1269             0.0        0.0  ...           0   
1         0.0          1430             0.0        0.0  ...           0   
2         0.0          1124             0.0        0.0  ...           0   
3         0.0          1184             0.0        0.0  ...           1   
4     

In [23]:
#hinzufügen der Märkte
marktdaten = pd.read_csv("./08_Maerkte/maekte_final.csv")
print(marktdaten.head())

#und mergen der Marktdaten
data = pd.merge(data, marktdaten, how='left', left_on='Datum', right_on='Datum')
print(data.head())

        Datum  Weihnachtsmarkt  Markt
0  2013-01-01                0      0
1  2013-01-02                0      0
2  2013-01-03                0      0
3  2013-01-04                0      0
4  2013-01-05                0      0
        Datum        Brot   Broetchen   Croissant  Konditorei      Kuchen  \
0  2013-07-01  148.828353  535.856285  201.198426   65.890169  317.475875   
1  2013-07-02  159.793757  546.780787  265.261254   74.543917  383.628682   
2  2013-07-03  111.885594  427.343259  210.260241   69.262728  305.523072   
3  2013-07-04  168.864941  454.859641  190.686641   61.490175  308.408168   
4  2013-07-05  171.280754  492.818804  181.644870   86.759861  355.518770   

   Saisonbrot  Umsatz_total  Temp_Very_Cold  Temp_Cold  ...  Samstag  Sonntag  \
0         0.0          1269             0.0        0.0  ...        0        0   
1         0.0          1430             0.0        0.0  ...        0        0   
2         0.0          1124             0.0        0.0  ...       

Damit sind alle Daten gemerged und der df kann als csv zur weiteren Bearbeitung exportiert werden.

In [24]:
#exportieren der Daten als csv
data.to_csv("data.csv", index=False)

In [26]:
#teilern der Daten in einen Trainingsdatensatz und einen Validierungsdatensatz
#trinaingsdaten sollen vom 01.07.2013 bis 31.07.20217 gehen
#validierungsdaten sollen vom 01.08.2017 bis 31.07.2018 gehen
train = data[data['Datum'] < '2017-08-01']
valid = data[data['Datum'] >= '2017-08-01']

print(train.shape)
print(valid.shape)

#exportieren der Daten als csv
train.to_csv("Trainingsdaten.csv", index=False)
valid.to_csv("Validierungsdaten.csv", index=False)



(1462, 40)
(357, 40)
